[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/open-atmos/PyPartMC-examples/blob/main/notebooks/lognorm_ex.ipynb)   
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PyPartMC-examples/blob/main/notebooks/lognorm_ex.ipynb)    
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PyPartMC-examples.git/main?urlpath=lab/tree/notebooks/lognorm_ex.ipynb)

In [1]:
# This file is a part of PyPartMC licensed under the GNU General Public License v3
# Copyright (C) 2022 University of Illinois Urbana-Champaign
# Authors: https://github.com/open-atmos/PyPartMC-examples/graphs/contributors

In [2]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install PyPartMC atmos_cloud_sim_uj_utils

In [3]:
from collections import namedtuple
import ipywidgets as widgets
import numpy as np
import PyPartMC as ppmc
from matplotlib import pyplot
from IPython.display import display, clear_output
from PyPartMC import si
from atmos_cloud_sim_uj_utils import show_plot

temperature_widget = widgets.FloatSlider(min=250, max=350)
temp_hbox = widgets.HBox([widgets.Label(value='Temperature [K]:'), temperature_widget])

fractal_widget = widgets.FloatSlider(min=1, max=3, value=3)
fractal_hbox = widgets.HBox([widgets.Label(value='Fractal Dimension []:'), fractal_widget])

vol_fill_widget = widgets.FloatSlider(min=1, max=1.5, value=1)
vol_hbox = widgets.HBox([widgets.Label(value='Volume Filling Factor []:'), vol_fill_widget])

mode_1_n_per_cc = widgets.IntSlider(min=0, max=100000, value=50000)
mode_1_n_hbox = widgets.HBox([widgets.Label(value='Mode 1 Number [#/cc]:'), mode_1_n_per_cc])
 
mode_1_gsd = widgets.FloatSlider(min=1.1, max=5, value=1.3)
mode_1_gsd_hbox = widgets.HBox([widgets.Label(value='Mode 1 Geometric Standard Deviation:'),
    mode_1_gsd])
 
mode_1_gm_microns = widgets.FloatSlider(min=0.001, max=10, value=0.9, readout_format='.3f')
mode_1_gm_hbox = widgets.HBox([widgets.Label(value='Mode 1 Geometric Mean Diameter [microns]:'),
    mode_1_gm_microns])

mode_2_n_per_cc = widgets.IntSlider(min=0, max=100000, value=80000)
mode_2_n_hbox = widgets.HBox([widgets.Label(value='Mode 2 Number [#/cc]:'), mode_2_n_per_cc])

mode_2_gsd = widgets.FloatSlider(min=1.1, max=5, value=2)
mode_2_gsd_hbox = widgets.HBox([widgets.Label(value='Mode 2 Geometric Standard Deviation:'),
    mode_2_gsd])

mode_2_gm_microns = widgets.FloatSlider(min=0.001, max=10, value=5.8, readout_format='.3f')
mode_2_gm_hbox = widgets.HBox([widgets.Label(value='Mode 2 Geometric Mean Diameter [microns]:'),
    mode_2_gm_microns])

button = widgets.Button(description='Calculate')
output = widgets.Output()

display(temp_hbox, fractal_hbox, vol_hbox, mode_1_n_hbox, mode_1_gsd_hbox, mode_1_gm_hbox,
       mode_2_n_hbox, mode_2_gsd_hbox, mode_2_gm_hbox, button, output)

def ln_norm():
    Mode = namedtuple("Mode", ("norm_factor", "geom_mean", "geom_stdev"))
    modes = (
        Mode(
            norm_factor=mode_1_n_per_cc.value/si.cm**3,
            geom_mean=mode_1_gm_microns.value*si.um,
            geom_stdev=mode_1_gsd.value
        ),
        Mode(
            norm_factor=mode_2_n_per_cc.value/si.cm**3,
            geom_mean=mode_2_gm_microns.value*si.um,
            geom_stdev=mode_2_gsd.value
        )
    )
    
    diameters = np.logspace(-1,1, 100) * si.um
    
    env_state = ppmc.EnvState({
        'rel_humidity': 0.,
        'latitude': 0.,
        'longitude': 0.,
        'altitude': 0.,
        'start_time': 0.,
        'start_day': 0
    })
    
    env_state.set_temperature(temperature_widget.value)
    
    def lognormal(diam, num, geom_mean, geom_stdev):
        return diam * (
            (num / (np.sqrt(2*np.pi)*diam*np.log(geom_stdev))) *
            np.exp(-(np.log(diam) - np.log(geom_mean))** 2 / (2*np.log(geom_stdev)** 2))
        )
    
    aero_data = ppmc.AeroData((
            {"H2O": [1000 * si.kg / si.m**3, 1, 18e-3 * si.kg / si.mol, 0]},
            {"Cl": [2200 * si.kg / si.m**3, 1, 35.5e-3 * si.kg / si.mol, 0]},
            {"Na": [2200 * si.kg / si.m**3, 1, 23e-3 * si.kg / si.mol, 0]}
        ))
    
    aero_data.frac_dim = fractal_widget.value
    aero_data.vol_fill_factor = vol_fill_widget.value
    
    volumes = (np.pi / 6) * diameters**3
    aero_particles = [ppmc.AeroParticle(aero_data, [0, volume, volume]) for volume in volumes]

    for aero_particle in aero_particles:
        ppmc.condense_equilib_particle(env_state, aero_data, aero_particle)

    wet_volumes = [particle.volumes[0] for particle in aero_particles]
    wet_diameters = ((6 / np.pi) * np.asarray(wet_volumes))**(1/3)
    
    with output:
        clear_output(wait=True)
        
        fig = pyplot.figure()
        
        fig.add_subplot(xscale='log')
        
        pyplot.plot(diameters, lognormal(diameters, *modes[0]))
        pyplot.plot(diameters, lognormal(diameters, *modes[1]))
        pyplot.plot(wet_diameters, lognormal(diameters, *modes[0]))
        pyplot.plot(wet_diameters, lognormal(diameters, *modes[1]))
        show_plot("spectrum.pdf")

def on_button_clicked(_):
    ln_norm()

button.on_click(on_button_clicked)

Button(description='Calculate', style=ButtonStyle())

Output()